In [1]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree
import copy

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.notebook import tqdm as tqdm
from scipy.signal import find_peaks

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

In [ ]:
!pip install cuDF

In [2]:
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [3]:
data = pd.read_csv('wtbdata_245days_normal_all.csv')

In [4]:
data

,index,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv,time_idx
0,0,1,1,00:00,6.17,-3.99,30.73,41.80,25.92,1.00,1.00,1.00,-0.25,494.66,1
1,1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.00,1.00,1.00,-0.25,494.66,2
2,2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.00,1.00,1.00,-0.24,509.76,3
3,3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.00,1.00,1.00,-0.26,542.53,4
4,4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.00,1.00,1.00,-0.23,509.36,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727515,4727515,134,245,23:10,7.79,2.80,-0.07,3.95,216.51,6.03,6.03,6.03,-111.69,601.44,35276
4727516,4727516,134,245,23:20,8.06,4.39,0.23,3.94,216.51,5.81,5.81,5.81,-72.67,673.79,35277
4727517,4727517,134,245,23:30,8.08,2.28,-0.16,4.15,216.51,0.68,0.68,0.68,-118.70,999.36,35278
4727518,4727518,134,245,23:40,8.46,0.80,-0.14,4.32,216.51,0.02,0.02,0.02,-58.12,1100.89,35279


In [5]:
def lag_feature(data, feat_columns, li):    
    # 在对应的位置设置lag列
    for shift_num in li:
        temp_data = data.groupby(['TurbID'])[feat_columns].shift(shift_num)
        temp_data.columns=[i+"_lag_"+str(shift_num) for i in feat_columns]
        data = pd.concat([data,temp_data],axis=1)
    return data 

In [6]:
def dataset_split(data):
    if 'Tmstamp' in data.columns.tolist():
        data = data.drop('Tmstamp',axis=1)
        
    train_data = data[lambda x: x.time_idx < 34993]
    val_data = data[(data.time_idx>=30816) & (data.time_idx < 34993)]
    test_data = data[lambda x: x.time_idx >= 34993]

    train_feat = train_data.drop('Patv',axis=1)
    train_label = train_data['Patv']
    val_feat = val_data.drop('Patv',axis=1)
    val_label = val_data['Patv']
    test_feat = test_data.drop('Patv',axis=1)
    test_label = test_data['Patv']
    test_individual_feat = []
    test_individual_label = []
    for i in range(data.TurbID.nunique()):
        tmp = test_data.drop('Patv',axis=1).loc[data['TurbID'] == i+1]
        tmp.columns = list(range(len(tmp.columns.tolist())))
        test_individual_feat.append(tmp)
        test_individual_label.append(test_data['Patv'].loc[data['TurbID'] == i+1])
    
    # 为了处理tsfresh的特殊符号列名，直接将所有列名写成数字
#     train_feat.columns = list(range(len(train_feat.columns.tolist())))
#     test_feat.columns = list(range(len(test_feat.columns.tolist())))
    
    return train_feat, train_label, val_feat, val_label, test_feat, test_label, test_individual_feat, test_individual_label

In [8]:
# from lightgbm import log_evaluation, early_stopping
# callbacks = [log_evaluation(period=100), early_stopping(stopping_rounds=20)]

In [9]:
def lgb_(train_feat, train_label, test_feat, test_label):
    gbm = LGBMRegressor(objective='regression', num_leaves=80, learning_rate=0.07, n_estimators=1000, max_depth=7)
    gbm.fit(train_feat, train_label, eval_set=[(test_feat, test_label)], eval_metric='rmse', early_stopping_rounds=20)
    return gbm

In [10]:
def lgb_test_new(gbm, data, test_feat, test_label, show=False):
    y_pred = gbm.predict(test_feat)
    patv_min = min(data.Patv)
    patv_max = max(data.Patv)
    for i in range(len(y_pred)):
        if y_pred[i] < 0:
            y_pred[i] = 0
        if y_pred[i] > patv_max:
            y_pred[i] = patv_max

    y_pred_clear = []
    test_label_clear = []
    for i in range(len(y_pred)):
#         if not (test_label.values[i] <= 0):
        y_pred_clear.append(y_pred[i])
        test_label_clear.append(test_label.values[i])
    
    
    # 模型评估
#     print('The rmse of prediction is:', mean_squared_error(test_label, y_pred) ** 0.5)
    pred_rmse = mean_squared_error(test_label_clear, y_pred_clear) ** 0.5
    print('The rmse of prediction is:', pred_rmse)

    # 模型评估
    pred_mae = mean_absolute_error(test_label_clear, y_pred_clear)
    print('The mae of prediction is:', pred_mae)

    # 特征重要度
#     print('{}\nFeature importances:{}'.format([column for column in test_feat], list(gbm.feature_importances_)))
    
    
    if show == True:
        plt.figure(figsize=(20,5))
        plt.plot(range(len(y_pred_clear)), y_pred_clear, label='pred')
        plt.plot(range(len(test_label_clear)), test_label_clear, label='label')
        plt.legend()
        plt.show()
    
    return y_pred_clear,pred_rmse,pred_mae

In [11]:
def get_test_feat(test_feat,test_label,time_idx):
    indexNames=test_feat[test_feat['time_idx'] !=time_idx].index
#     print(indexNames)
    test_feat = test_feat.drop(indexNames)
    test_label= test_label.drop(indexNames)
    return test_feat,test_label

In [12]:
def lag_insert(test_feat, num,index_list): #构造test集函数
    
    for j in range(1,num+1):
        
        time_idx_list = [m for m in range(34993,34993+j)]

        print(time_idx_list)

        feature_list = test_feat.columns[4+j -1:4+j]
        print(feature_list)


        for i in range(len(index_list)):
            for feature in feature_list:
                if test_feat.loc[index_list[i],'time_idx'] in time_idx_list:
                    test_feat.loc[index_list[i],feature] = data.loc[index_list[i]-j,'Patv']

                else:
                    test_feat.loc[index_list[i],feature] = np.nan

    return test_feat

In [13]:
def round_pred(gbm1,test_feat,test_label,pred_1,pred_list,rmse_list,mae_list,index_list): #循环预测函数
    
    for i in range(34994,34994+287):
        feature_list = test_feat.columns[4:]
        flag = 0
        for j in range(len(index_list)): #用预测的值填充lag1-n

            if test_feat.loc[index_list[j],'time_idx'] == i:

                for f in range(len(feature_list)):

                    if (index_list[j]+f) in index_list: #边界条件判断是否溢出

    #                     print(index_list[j]+flag)
                        test_feat.loc[index_list[j]+f,feature_list[f]] = pred_1[flag]

                flag += 1
#     print(exp_feat)
#     break;
        real_test_feat,real_test_label = get_test_feat(test_feat,test_label,i)
        print('predict:{}/288'.format(i-34993+1))
        pred_1,pred_rmse,pred_mae = lgb_test_new(gbm1, data, real_test_feat, real_test_label)
        print('----------------------------')
        pred_list.extend(pred_1)
        rmse_list.append(pred_rmse)
        mae_list.append(pred_mae)
    return pred_list,rmse_list,mae_list

In [14]:
def single_step(lag_n):
    # 做lag特征
    feat_columns = ['Patv']
    li = [i for i in range(1,lag_n+1)]
    data_lag1 = lag_feature(data, feat_columns, li)
    data_lag1 = data_lag1.drop(['Wspd','Wdir','Etmp','Itmp','Ndir','Pab1','Pab2','Pab3','Prtv'],axis = 1)
#     data_lag1
    
    #分割数据集
    train_feat, train_label, val_feat, val_label, test_feat, test_label, test_individual_feat, test_individual_label = dataset_split(data_lag1)
    
    # 模型训练
    gbm1 = lgb_(train_feat, train_label, val_feat, val_label)
    
    #获取修改值的index
    index_list = list(test_feat['index'])
    
    #构造单步预测的test集
    test_feat = lag_insert(test_feat, lag_n,index_list)
#     print(test_feat)
    
    #取出288时刻中的第一个时刻进行预测
    real_test_feat,real_test_label = get_test_feat(test_feat,test_label,34993)
    
    #保存信息的list
    pred_list,rmse_list, mae_list = [], [], []
    
    #第一个时刻进行预测，pred_1的长度为134，即134个风机的第一个时刻的预测
    pred_1,pred_rmse,pred_mae = lgb_test_new(gbm1, data, real_test_feat, real_test_label)
    
    #记录预测以及误差
    pred_list.extend(pred_1)
    rmse_list.append(pred_rmse)
    mae_list.append(pred_mae)
    
    #循环预测
    print('开始循环预测')
    pred_list,rmse_list,mae_list = round_pred(gbm1,test_feat,test_label,pred_1,pred_list, rmse_list, mae_list,index_list)
    
    return pred_list,rmse_list,mae_list
    

In [19]:
pred_list,rmse_list,mae_list = single_step(100)

[1]	valid_0's rmse: 328.945	valid_0's l2: 108205
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 307.08	valid_0's l2: 94298.3
[3]	valid_0's rmse: 286.794	valid_0's l2: 82250.7
[4]	valid_0's rmse: 268.008	valid_0's l2: 71828.2
[5]	valid_0's rmse: 250.601	valid_0's l2: 62801
[6]	valid_0's rmse: 234.504	valid_0's l2: 54992.1
[7]	valid_0's rmse: 219.596	valid_0's l2: 48222.4
[8]	valid_0's rmse: 205.71	valid_0's l2: 42316.7
[9]	valid_0's rmse: 192.992	valid_0's l2: 37246
[10]	valid_0's rmse: 181.259	valid_0's l2: 32855
[11]	valid_0's rmse: 170.406	valid_0's l2: 29038.2
[12]	valid_0's rmse: 160.338	valid_0's l2: 25708.4
[13]	valid_0's rmse: 151.097	valid_0's l2: 22830.4
[14]	valid_0's rmse: 142.653	valid_0's l2: 20350
[15]	valid_0's rmse: 134.893	valid_0's l2: 18196.2
[16]	valid_0's rmse: 127.732	valid_0's l2: 16315.5
[17]	valid_0's rmse: 121.225	valid_0's l2: 14695.5
[18]	valid_0's rmse: 115.321	valid_0's l2: 13298.8
[19]	valid_0's rmse: 109.914	valid_0's l2

[162]	valid_0's rmse: 61.4921	valid_0's l2: 3781.28
[163]	valid_0's rmse: 61.467	valid_0's l2: 3778.2
[164]	valid_0's rmse: 61.2856	valid_0's l2: 3755.92
[165]	valid_0's rmse: 61.2839	valid_0's l2: 3755.71
[166]	valid_0's rmse: 61.2816	valid_0's l2: 3755.44
[167]	valid_0's rmse: 61.2807	valid_0's l2: 3755.32
[168]	valid_0's rmse: 61.2795	valid_0's l2: 3755.18
[169]	valid_0's rmse: 61.2795	valid_0's l2: 3755.18
[170]	valid_0's rmse: 61.2782	valid_0's l2: 3755.02
[171]	valid_0's rmse: 61.2756	valid_0's l2: 3754.7
[172]	valid_0's rmse: 61.2751	valid_0's l2: 3754.64
[173]	valid_0's rmse: 61.2739	valid_0's l2: 3754.49
[174]	valid_0's rmse: 61.2701	valid_0's l2: 3754.02
[175]	valid_0's rmse: 61.27	valid_0's l2: 3754.01
[176]	valid_0's rmse: 61.2651	valid_0's l2: 3753.41
[177]	valid_0's rmse: 61.2423	valid_0's l2: 3750.62
[178]	valid_0's rmse: 61.2422	valid_0's l2: 3750.61
[179]	valid_0's rmse: 61.2393	valid_0's l2: 3750.25
[180]	valid_0's rmse: 61.2351	valid_0's l2: 3749.74
[181]	valid_0's r

[321]	valid_0's rmse: 60.4146	valid_0's l2: 3649.93
[322]	valid_0's rmse: 60.4094	valid_0's l2: 3649.29
[323]	valid_0's rmse: 60.4094	valid_0's l2: 3649.29
[324]	valid_0's rmse: 60.4051	valid_0's l2: 3648.78
[325]	valid_0's rmse: 60.4041	valid_0's l2: 3648.65
[326]	valid_0's rmse: 60.3996	valid_0's l2: 3648.11
[327]	valid_0's rmse: 60.3992	valid_0's l2: 3648.06
[328]	valid_0's rmse: 60.3993	valid_0's l2: 3648.07
[329]	valid_0's rmse: 60.3097	valid_0's l2: 3637.26
[330]	valid_0's rmse: 60.3091	valid_0's l2: 3637.18
[331]	valid_0's rmse: 60.3086	valid_0's l2: 3637.13
[332]	valid_0's rmse: 60.3075	valid_0's l2: 3637
[333]	valid_0's rmse: 60.3075	valid_0's l2: 3637
[334]	valid_0's rmse: 60.3037	valid_0's l2: 3636.54
[335]	valid_0's rmse: 60.3007	valid_0's l2: 3636.17
[336]	valid_0's rmse: 60.3007	valid_0's l2: 3636.17
[337]	valid_0's rmse: 60.2937	valid_0's l2: 3635.33
[338]	valid_0's rmse: 60.2919	valid_0's l2: 3635.12
[339]	valid_0's rmse: 60.2907	valid_0's l2: 3634.97
[340]	valid_0's rm

[480]	valid_0's rmse: 59.4116	valid_0's l2: 3529.74
[481]	valid_0's rmse: 59.4077	valid_0's l2: 3529.27
[482]	valid_0's rmse: 59.407	valid_0's l2: 3529.19
[483]	valid_0's rmse: 59.4053	valid_0's l2: 3528.99
[484]	valid_0's rmse: 59.4049	valid_0's l2: 3528.94
[485]	valid_0's rmse: 59.4037	valid_0's l2: 3528.8
[486]	valid_0's rmse: 59.4025	valid_0's l2: 3528.65
[487]	valid_0's rmse: 59.4024	valid_0's l2: 3528.65
[488]	valid_0's rmse: 59.4023	valid_0's l2: 3528.63
[489]	valid_0's rmse: 59.4008	valid_0's l2: 3528.46
[490]	valid_0's rmse: 59.4003	valid_0's l2: 3528.4
[491]	valid_0's rmse: 59.3896	valid_0's l2: 3527.13
[492]	valid_0's rmse: 59.3871	valid_0's l2: 3526.83
[493]	valid_0's rmse: 59.3871	valid_0's l2: 3526.83
[494]	valid_0's rmse: 59.3843	valid_0's l2: 3526.49
[495]	valid_0's rmse: 59.3843	valid_0's l2: 3526.49
[496]	valid_0's rmse: 59.3836	valid_0's l2: 3526.41
[497]	valid_0's rmse: 59.3818	valid_0's l2: 3526.2
[498]	valid_0's rmse: 59.3779	valid_0's l2: 3525.73
[499]	valid_0's 

[34993, 34994, 34995, 34996, 34997, 34998, 34999, 35000, 35001, 35002, 35003, 35004, 35005, 35006, 35007, 35008, 35009, 35010, 35011, 35012, 35013, 35014, 35015, 35016, 35017, 35018, 35019, 35020, 35021, 35022, 35023, 35024, 35025, 35026, 35027, 35028, 35029, 35030, 35031, 35032]
Index(['Patv_lag_40'], dtype='object')
[34993, 34994, 34995, 34996, 34997, 34998, 34999, 35000, 35001, 35002, 35003, 35004, 35005, 35006, 35007, 35008, 35009, 35010, 35011, 35012, 35013, 35014, 35015, 35016, 35017, 35018, 35019, 35020, 35021, 35022, 35023, 35024, 35025, 35026, 35027, 35028, 35029, 35030, 35031, 35032, 35033]
Index(['Patv_lag_41'], dtype='object')
[34993, 34994, 34995, 34996, 34997, 34998, 34999, 35000, 35001, 35002, 35003, 35004, 35005, 35006, 35007, 35008, 35009, 35010, 35011, 35012, 35013, 35014, 35015, 35016, 35017, 35018, 35019, 35020, 35021, 35022, 35023, 35024, 35025, 35026, 35027, 35028, 35029, 35030, 35031, 35032, 35033, 35034]
Index(['Patv_lag_42'], dtype='object')
[34993, 34994, 3499

...

The rmse of prediction is: 99.40192798279982
The mae of prediction is: 69.13165686466853
开始循环预测
predict:2/288
The rmse of prediction is: 142.9723377812963
The mae of prediction is: 101.73975826084295
----------------------------
predict:3/288
The rmse of prediction is: 155.7966242409817
The mae of prediction is: 101.83736865065929
----------------------------
predict:4/288
The rmse of prediction is: 163.90728980826964
The mae of prediction is: 108.14697803528676
----------------------------
predict:5/288
The rmse of prediction is: 173.3130072092343
The mae of prediction is: 120.16144137918671
----------------------------
predict:6/288
The rmse of prediction is: 167.68211565443536
The mae of prediction is: 118.88955219892532
----------------------------
predict:7/288
The rmse of prediction is: 159.4480943949505
The mae of prediction is: 114.8872229927621
----------------------------
predict:8/288
The rmse of prediction is: 147.6285181739485
The mae of prediction is: 112.67071908540392
-

predict:63/288
The rmse of prediction is: 279.349354032903
The mae of prediction is: 223.3355662915059
----------------------------
predict:64/288
The rmse of prediction is: 292.0060461745557
The mae of prediction is: 237.17457556743537
----------------------------
predict:65/288
The rmse of prediction is: 289.80563371773724
The mae of prediction is: 226.610481375121
----------------------------
predict:66/288
The rmse of prediction is: 297.2236719959306
The mae of prediction is: 247.0633202939719
----------------------------
predict:67/288
The rmse of prediction is: 315.6118081253127
The mae of prediction is: 276.0429410692493
----------------------------
predict:68/288
The rmse of prediction is: 325.46074574031326
The mae of prediction is: 286.02367177910287
----------------------------
predict:69/288
The rmse of prediction is: 309.88725366591177
The mae of prediction is: 261.51167342329694
----------------------------
predict:70/288
The rmse of prediction is: 313.86189562987073
The 

The rmse of prediction is: 1073.5988095708897
The mae of prediction is: 1071.051705487198
----------------------------
predict:125/288
The rmse of prediction is: 1105.1348944024153
The mae of prediction is: 1102.6772731159015
----------------------------
predict:126/288
The rmse of prediction is: 1138.5566536498754
The mae of prediction is: 1136.463145232218
----------------------------
predict:127/288
The rmse of prediction is: 1156.1996416447555
The mae of prediction is: 1154.4987784023415
----------------------------
predict:128/288
The rmse of prediction is: 1169.6965520709655
The mae of prediction is: 1168.4038156048382
----------------------------
predict:129/288
The rmse of prediction is: 1175.465752460992
The mae of prediction is: 1174.293895879508
----------------------------
predict:130/288
The rmse of prediction is: 1183.1124987242058
The mae of prediction is: 1182.1478383948488
----------------------------
predict:131/288
The rmse of prediction is: 1189.0974942524804
The ma

predict:185/288
The rmse of prediction is: 1137.8353388152523
The mae of prediction is: 1136.228505684669
----------------------------
predict:186/288
The rmse of prediction is: 1138.614507762861
The mae of prediction is: 1136.2205153582536
----------------------------
predict:187/288
The rmse of prediction is: 1153.7738602346062
The mae of prediction is: 1152.5778833947188
----------------------------
predict:188/288
The rmse of prediction is: 1160.725192774492
The mae of prediction is: 1159.747728952814
----------------------------
predict:189/288
The rmse of prediction is: 1130.7911109605038
The mae of prediction is: 1128.7252975881008
----------------------------
predict:190/288
The rmse of prediction is: 1090.4148217600525
The mae of prediction is: 1084.3570281075438
----------------------------
predict:191/288
The rmse of prediction is: 1044.5446489597928
The mae of prediction is: 1030.1245940004733
----------------------------
predict:192/288
The rmse of prediction is: 1030.5791

predict:246/288
The rmse of prediction is: 337.28860188619706
The mae of prediction is: 300.028729857858
----------------------------
predict:247/288
The rmse of prediction is: 350.7747686808191
The mae of prediction is: 302.95492679161964
----------------------------
predict:248/288
The rmse of prediction is: 351.7652170689964
The mae of prediction is: 300.79761413764425
----------------------------
predict:249/288
The rmse of prediction is: 370.66216153420334
The mae of prediction is: 328.0934455017192
----------------------------
predict:250/288
The rmse of prediction is: 371.4150014054581
The mae of prediction is: 332.0430097843048
----------------------------
predict:251/288
The rmse of prediction is: 400.385711198482
The mae of prediction is: 366.0543676123401
----------------------------
predict:252/288
The rmse of prediction is: 422.13515504668163
The mae of prediction is: 384.47562675728113
----------------------------
predict:253/288
The rmse of prediction is: 432.20401424408